# 고객 데이터에서 숨겨진 패턴 찾기: PCA와 클러스터링

지난 시간, 우리는 통계분석의 다양한 방법들을 배웠습니다. 특히, 변수가 많은 고차원 데이터는 해석하기 어렵고, 분석 모델의 성능을 저하시키는 '차원의 저주' 문제가 발생할 수 있다는 점을 확인했습니다. 또한, 정답이 없는 데이터 속에서 숨겨진 구조나 그룹을 찾아내는 비지도 학습의 중요성도 배웠습니다.

오늘은 **주성분 분석(PCA)**을 이용해 복잡한 고객 데이터를 핵심 정보만 남기고 차원을 축소하고, **K-평균(K-Means) 군집분석**을 통해 고객들을 의미 있는 그룹으로 나누는 과정을 직접 코드로 구현합니다. 데이터의 숨겨진 인사이트를 발견하고, 이를 바탕으로 비즈니스 의사결정을 내리는 데이터 분석가의 핵심 역량을 체험하게 될 것입니다.

**🎯 프로젝트 목표:**

1.  **차원 축소의 이해**: '차원의 저주'가 무엇인지 이해하고, **주성분 분석(PCA)**을 통해 고차원 데이터를 시각화 및 분석이 용이한 저차원 데이터로 변환합니다.
2.  **주성분 해석**: PCA로 생성된 새로운 축(주성분)이 원래 데이터의 어떤 특성을 요약하는지 **주성분 부하량(Loadings)**을 통해 해석하는 능력을 기릅니다.
3.  **군집분석 적용**: 정답 없는 데이터에서 자연스러운 그룹을 찾아내는 **K-평균(K-Means) 군집분석**의 원리를 이해하고, 최적의 군집 개수(k)를 찾는 **엘보우 방법(Elbow Method)**을 적용합니다.
4.  **인사이트 도출 및 의사결정**: 차원 축소와 군집분석 결과를 종합하여 각 고객 그룹의 특징(페르소나)을 정의하고, 실제 비즈니스에 적용할 수 있는 마케팅 전략을 제안합니다.


## 🔬 실습 진행 안내

오늘 실습은 다음 순서로 진행됩니다. 각 단계의 목표를 명확히 인지하고 참여해 주세요.

1.  **개인 학습 (Individual Work):**

    - 먼저 **문제 1부터 4까지 모든 문제를 개인적으로** 풀어봅니다. 코드의 빈칸을 채우고, 각 문제의 핵심 개념과 '생각해 볼 문제'에 대한 자신의 답을 정리합니다.
    - 문제들이 서로 연결되어 있으므로(예: 문제 1의 결과는 문제 3에서 사용), 반드시 순서대로 모든 문제를 풀어야 합니다.

2.  **팀별 심화 및 발표 준비 (Team Deep-dive & Preparation):**

    - 각 팀은 **지정된 문제 하나**를 맡아 해당 문제의 '전문가'가 됩니다.
    - 개인적으로 풀었던 내용을 바탕으로, 팀원들과 함께 맡은 문제의 핵심 개념, 소스 코드, 생각해 볼 문제에 대한 답을 더 깊이 있게 토론하고 정리합니다.
    - 각 문제의 마지막에 있는 **'[발표 과제]'** 가이드를 참고하여 발표를 준비합니다.

3.  **팀별 발표 (Team Presentation):**
    - 각 팀은 맡은 문제에 대해 다른 모든 학생들을 가르쳐준다는 생각으로 발표를 진행합니다.
    - 문제의 배경, 핵심 개념, 코드 실행 과정, 결과 해석, 그리고 '생각해 볼 문제'에 대한 팀의 결론을 명확하게 전달해야 합니다.


## 📖 시작하기 전: 핵심 용어 사전

오늘 실습에서 계속 마주칠 핵심 용어들입니다. '이런 게 있구나' 정도로 가볍게 읽고 넘어가세요. 실습을 진행하며 각 용어의 의미가 더 명확해질 것입니다.

- **차원(Dimension):** 데이터가 가진 변수(피처)의 개수. (예: 고객 데이터에 '나이', '소득', '방문 빈도'가 있다면 3차원 데이터)
- **차원의 저주(Curse of Dimensionality):** 차원이 증가할수록 데이터 분석 및 모델링에 필요한 데이터 양이 기하급수적으로 증가하고, 데이터 간의 거리 측정이 어려워지는 현상.
- **주성분 분석(PCA):** 여러 변수 간에 존재하는 상관관계를 이용해, 데이터의 분산(정보)을 가장 잘 설명하는 새로운 축(주성분)들을 찾아 기존 데이터를 요약/축소하는 기법.
- **주성분(Principal Component):** PCA를 통해 생성된 새로운 변수. 첫 번째 주성분(PC1)이 데이터의 분산을 가장 많이 설명하며, 각 주성분은 서로 직교(독립)합니다.
- **기여율(Explained Variance Ratio):** 각 주성분이 전체 데이터 분산의 몇 %를 설명하는지를 나타내는 지표.
- **군집분석(Clustering):** 데이터의 유사성을 기반으로 전체 데이터를 몇 개의 그룹(군집)으로 나누는 비지도 학습 방법.
- **K-평균(K-Means):** 분석가가 지정한 k개의 중심점(centroid)을 기준으로, 각 데이터 포인트를 가장 가까운 중심점에 할당하는 과정을 반복하여 군집을 형성하는 알고리즘.
- **엘보우 방법(Elbow Method):** K-평균 군집분석에서 군집의 개수(k)를 늘려가면서, 각 군집 내 데이터들의 응집도(inertia) 변화를 그래프로 그려 최적의 k를 찾는 방법.


## 사전 준비: 라이브러리 및 데이터 로드

오늘은 데이터 분석을 위한 `pandas`, `numpy`, 시각화를 위한 `matplotlib`, `seaborn`, 그리고 머신러닝을 위한 `scikit-learn` 라이브러리를 사용합니다.


In [ ]:
# 필요 라이브러리 설치
# !pip install scikit-learn seaborn

# 코랩에서 한글 폰트 사용을 위한 설정 (필요 시 주석 해제 후 실행)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 위 코드 실행 후 상단 메뉴에서 [런타임] > [런타임 다시 시작]을 눌러주세요.

In [ ]:
# 런타임 다시 시작 후, 이 셀을 실행하여 라이브러리와 폰트를 로드합니다.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# 한글 폰트 설정 (Mac 사용자는 'AppleGothic', Windows 사용자는 'Malgun Gothic' 등)
plt.rc('font', family='NanumGothic')
# plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 실습용 데이터 생성
np.random.seed(42)
data = pd.DataFrame({
    '연간소득': np.random.normal(5000, 1500, 200).astype(int),
    '소비점수': np.random.randint(1, 101, 200),
    '방문빈도': np.random.randint(1, 101, 200),
    '체류시간': np.random.normal(60, 20, 200).astype(int)
})
print("데이터 생성 완료!")
data.head()

---

## 🚀 프로젝트 시나리오: "쇼핑몰 고객, 어떻게 그룹화할까?"

우리 팀은 쇼핑몰 고객 데이터를 확보했습니다. 데이터에는 고객의 **'연간소득', '소비점수(1~100점)', '월평균 방문빈도', '평균 체류시간(분)'** 이라는 4가지 변수가 있습니다. 이 4차원 데이터를 이용해 고객들을 비슷한 그룹으로 묶고, 각 그룹의 특성에 맞는 맞춤형 마케팅 전략을 세우고자 합니다.

하지만 4개의 변수를 한 번에 고려하여 고객을 나누는 것은 직관적이지 않고 어렵습니다. 과연 우리는 이 복잡한 데이터를 효과적으로 요약하고, 그 안에 숨겨진 고객 그룹들을 찾아낼 수 있을까요?


## 문제 1 (난이도: 하): 데이터 탐색과 차원 축소의 필요성 이해하기

> **🎯 목표:** 고차원 데이터의 시각적 탐색 한계를 확인하고, PCA를 적용하기 위한 데이터 전처리(스케일링)와 차원 축소를 수행합니다.


### 💡 핵심 개념:

**주성분 분석(PCA)**은 변수의 스케일에 민감합니다. 예를 들어 '연간소득'처럼 값의 범위가 큰 변수가 다른 변수들(예: '소비점수')보다 분석에 더 큰 영향을 미칠 수 있습니다. 이를 방지하기 위해, PCA 적용 전에 각 변수의 평균을 0, 분산을 1로 맞추는 **표준화(Standardization)** 과정을 거치는 것이 일반적입니다. `StandardScaler`가 이 역할을 수행합니다.


### 📌 수행 과제:

1.  `seaborn`의 `pairplot`을 이용해 4개 변수 간의 관계를 시각화하고, 이로부터 고객 그룹을 직관적으로 파악하기 어려운 이유를 생각해봅니다.
2.  `StandardScaler`를 사용하여 데이터를 표준화합니다.
3.  `PCA` 모델을 `n_components=2`로 설정하여 4차원 데이터를 2차원으로 축소합니다.
4.  축소된 2차원 데이터를 산점도(scatter plot)로 시각화하여 데이터가 어떻게 분포하는지 확인합니다.


In [ ]:
# 1. Pairplot으로 데이터 탐색
sns.pairplot(data)
plt.show()

# 2. 데이터 표준화 (Standard Scaling)
scaler = StandardScaler()
# TODO-1: data를 scaler에 맞추고(fit) 변환(transform)하여 scaled_data에 저장하세요.
scaled_data = scaler.fit_transform(____)

# 3. PCA 수행
# TODO-2: n_components를 2로 설정하여 PCA 모델을 초기화하세요.
pca = PCA(n_components=___)
pca_data = pca.fit_transform(scaled_data)

# 4. PCA 결과 시각화
pca_df = pd.DataFrame(data=pca_data, columns=['PC1', 'PC2'])

plt.figure(figsize=(8, 6))
plt.scatter(pca_df['PC1'], pca_df['PC2'], alpha=0.7)
plt.title('PCA를 이용한 2차원 데이터 시각화')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

# --- 해석 ---
# Pairplot과 PCA 산점도를 비교했을 때, 어떤 차이가 있나요? 왜 PCA가 유용한가요? (주석에 직접 작성해보세요)

### 🤔 생각해 볼 문제:

1.  PCA를 통해 4차원 데이터가 2차원으로 축소되었습니다. 이 과정에서 정보의 손실이 발생했을까요? `pca.explained_variance_ratio_` 속성을 출력하고, PC1과 PC2가 전체 데이터 분산의 몇 %를 각각 설명하는지 확인해보세요. 두 주성분을 합치면 총 몇 %의 정보를 보존하고 있나요?
2.  만약 데이터 표준화(Scaling) 과정을 생략하고 PCA를 적용한다면 어떤 결과가 나올까요? '연간소득' 변수가 결과에 어떤 영향을 미칠지 예상해보세요.
3.  2차원으로 축소된 산점도를 보니, 데이터들이 몇 개의 덩어리(군집)로 나뉠 수 있을 것처럼 보이나요? 대략 몇 개의 그룹으로 나눌 수 있을지 예상해보세요.

---

### 📢 **[발표 과제]**

문제 1을 맡은 팀은 **데이터 전처리 및 PCA 기본 적용**의 전문가가 되어 다음 내용을 중심으로 발표를 준비하세요.

1.  **핵심 개념 설명:** '차원의 저주'를 간단한 예시로 설명하고, 왜 PCA와 같은 차원 축소가 필요한지 설명합니다. PCA 전 데이터 스케일링의 중요성을 강조하세요.
2.  **코드 리뷰:** `StandardScaler`와 `PCA`를 사용하는 과정을 단계별로 설명합니다. Pairplot과 PCA 산점도 결과를 비교하며, PCA가 어떻게 복잡한 데이터를 한눈에 파악하기 쉽게 만드는지 보여줍니다.
3.  **'생각해 볼 문제' 토의:** 위 3가지 질문에 대한 팀의 답변을 공유합니다. 특히 `explained_variance_ratio_`를 해석하는 방법을 통해, PCA가 '정보 손실'과 '데이터 요약' 사이의 트레이드오프 관계에 있음을 설명해 보세요.


## 문제 2 (난이도: 중): 주성분 의미 해석하기

> **🎯 목표:** PCA로 생성된 주성분(PC1, PC2)이 원래 변수들과 어떤 관계를 갖는지 분석하여, 각 주성분에 의미를 부여합니다.


### 💡 핵심 개념:

**주성분 부하량(Principal Component Loadings)**은 원래 변수와 주성분 간의 상관계수를 나타냅니다. 특정 변수의 부하량 절댓값이 크다면, 그 변수가 해당 주성분을 형성하는 데 중요한 역할을 한다는 의미입니다. `pca.components_` 속성을 통해 이 부하량을 확인할 수 있습니다. 이를 통해 우리는 'PC1은 소득과 관련된 축', 'PC2는 쇼핑 스타일과 관련된 축'과 같이 주성분의 의미를 해석할 수 있습니다.


### 📌 수행 과제:

1.  `pca.components_` 속성을 확인하여 주성분 부하량을 담은 데이터프레임을 생성합니다. (index는 'PC1', 'PC2', columns는 원래 변수명)
2.  `seaborn`의 `heatmap`을 이용해 부하량을 시각화하여 어떤 변수가 각 주성분에 큰 영향을 미치는지 한눈에 파악합니다.
3.  부하량 값을 바탕으로 PC1과 PC2가 각각 어떤 고객 특성을 대표하는지 해석해봅니다.


In [ ]:
# 1. 주성분 부하량(Loadings) 확인
# TODO-1: pca.components_와 data.columns를 사용하여 부하량 데이터프레임(loadings_df)을 만드세요.
loadings_df = pd.DataFrame(
    data=___,
    columns=___,
    index=['PC1', 'PC2']
)
print("주성분 부하량:")
print(loadings_df)

# 2. 히트맵으로 부하량 시각화
plt.figure(figsize=(10, 3))
sns.heatmap(loadings_df, annot=True, cmap='viridis', fmt='.2f')
plt.title('주성분 부하량 히트맵')
plt.show()

# 3. 주성분 해석 (아래 주석에 직접 작성해 보세요)
# PC1의 부하량을 보면, 어떤 변수들의 값이 양수/음수로 높은가요? PC1이 높다는 것은 무엇을 의미할까요?
# PC1 의미 해석:
# PC2의 부하량을 보면, 어떤 변수들의 값이 양수/음수로 높은가요? PC2가 높다는 것은 무엇을 의미할까요?
# PC2 의미 해석:

### 🤔 생각해 볼 문제:

1.  문제 1에서 그린 PCA 산점도에서 x축(PC1)의 오른쪽으로 갈수록, y축(PC2)의 위쪽으로 갈수록 어떤 특성을 가진 고객들이 위치할 것이라고 예상할 수 있나요? 여러분이 해석한 주성분의 의미와 연결지어 설명해보세요.
2.  '연간소득'과 '체류시간' 변수는 PC1에 미치는 영향(부하량의 부호)이 서로 비슷했나요, 달랐나요? 이는 두 변수 간에 어떤 관계가 있음을 시사할까요?
3.  만약 새로운 주성분 PC3를 추가로 분석한다면, PC3는 PC1이나 PC2가 설명하지 못한 데이터의 어떤 또 다른 특성을 잡아낼 것이라고 기대할 수 있을까요?

---

### 📢 **[발표 과제]**

문제 2를 맡은 팀은 **주성분 해석**의 전문가가 되어 다음 내용을 중심으로 발표를 준비하세요.

1.  **핵심 개념 설명:** 주성분 부하량(Loadings)이 무엇인지, 그리고 이 값이 왜 주성분의 '정체'를 파악하는 데 핵심적인지 설명합니다.
2.  **코드 리뷰 및 결과 해석:** `pca.components_`를 데이터프레임과 히트맵으로 시각화하는 과정을 보여줍니다. 히트맵의 색상과 숫자를 보며 PC1과 PC2가 각각 어떤 원래 변수들과 강하게 연관되어 있는지, 그래서 각 축에 어떤 의미(예: '경제력 축', '쇼핑 열정 축')를 부여할 수 있는지 논리적으로 설명합니다.
3.  **'생각해 볼 문제' 토의:** 위 3가지 질문에 대한 팀의 답변을 발표합니다. 특히 PCA 산점도의 각 사분면에 위치할 고객들의 페르소나를 추측해보며, 주성분 해석이 어떻게 데이터에 대한 깊이 있는 이해로 이어지는지 보여주세요.


## 문제 3 (난이도: 상): K-평균 군집분석으로 고객 그룹 찾기

> **🎯 목표:** K-평균 군집분석을 PCA로 차원 축소된 데이터에 적용하여 고객들을 자동으로 그룹화하고, 엘보우 방법을 이용해 최적의 군집 개수(k)를 결정합니다.


### 💡 핵심 개념:

K-평균 알고리즘을 사용하려면 '몇 개의 군집으로 나눌지(k)'를 미리 정해야 합니다. **엘보우 방법(Elbow Method)**은 k를 1부터 점차 늘려가며 각 k에 대한 군집 결과를 평가하는 방법입니다. 평가 지표로는 군집 내 데이터들이 중심에 얼마나 잘 뭉쳐있는지를 나타내는 **이너셔(Inertia)** 값을 사용합니다. k가 증가할수록 이너셔는 계속 감소하는데, 이 감소율이 급격히 완만해지는 지점(그래프 모양이 팔꿈치처럼 꺾이는 부분)을 최적의 k로 선택합니다.


### 📌 수행 과제:

1.  k를 1부터 10까지 변화시키면서 K-평균 군집분석을 반복 수행하고, 각 k에 대한 이너셔(`kmeans.inertia_`) 값을 리스트에 저장합니다.
2.  저장된 이너셔 값을 꺾은선 그래프로 시각화하여 '팔꿈치'에 해당하는 최적의 k를 찾습니다.
3.  찾아낸 최적의 k를 사용하여 K-평균 모델을 다시 학습시키고, 각 데이터 포인트에 대한 군집 레이블을 얻습니다.
4.  PCA 산점도에 군집 레이블에 따라 다른 색상을 적용하여 군집화 결과를 시각화합니다.


In [ ]:
# 1. 엘보우 방법을 위한 이너셔(Inertia) 계산
inertia_list = []
k_range = range(1, 11)

for k in k_range:
    # TODO-1: n_clusters를 k로, random_state를 42로 설정하여 KMeans 모델을 초기화하세요.
    kmeans = KMeans(n_clusters=___, random_state=42, n_init=10)
    kmeans.fit(pca_data)
    # TODO-2: 학습된 kmeans 모델의 inertia_ 속성을 inertia_list에 추가하세요.
    inertia_list.append(___)

# 2. 엘보우 그래프 시각화
plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia_list, marker='o')
plt.title('엘보우 방법을 이용한 최적의 k 찾기')
plt.xlabel('군집의 개수 (k)')
plt.ylabel('이너셔 (Inertia)')
plt.xticks(k_range)
plt.grid(True)
plt.show()

# 3. 최적의 k로 K-Means 다시 실행
# TODO-3: 위 그래프를 보고 최적의 k값을 결정하여 optimal_k 변수에 할당하세요.
optimal_k = ___
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(pca_data)

# 4. 군집화 결과 시각화
pca_df['cluster'] = cluster_labels
plt.figure(figsize=(10, 7))
sns.scatterplot(x='PC1', y='PC2', hue='cluster', data=pca_df, palette='viridis', s=100, alpha=0.8)
plt.title(f'{optimal_k}개 군집으로 나눈 고객 분포')
plt.grid(True)
plt.legend(title='Cluster')
plt.show()

### 🤔 생각해 볼 문제:

1.  엘보우 그래프에서 왜 '팔꿈치' 지점이 최적의 k가 되는 것일까요? k를 그보다 더 크게 설정하면 어떤 장단점이 있을까요?
2.  군집분석을 원본 데이터(4차원)가 아닌 PCA 데이터(2차원)에 적용했습니다. 이렇게 하는 것의 장점은 무엇일까요? 만약 원본 데이터에 직접 K-평균을 적용했다면 결과가 어떻게 달라졌을지 예상해보세요.
3.  시각화된 군집 결과를 보면, 군집들이 잘 나뉘었다고 생각하나요? 경계가 모호한 데이터 포인트들은 왜 그렇게 분류되었을지, PCA 산점도와 연관지어 생각해보세요.

---

### 📢 **[발표 과제]**

문제 3을 맡은 팀은 **K-평균 군집분석**의 전문가가 되어 다음 내용을 중심으로 발표를 준비하세요.

1.  **핵심 개념 설명:** K-평균 알고리즘의 동작 원리를 간단한 그림이나 비유로 설명합니다. '최적의 k'를 찾는 것이 왜 중요한 문제인지, 그리고 엘보우 방법이 어떻게 그 해답을 주는지 이너셔(Inertia) 개념과 함께 설명하세요.
2.  **코드 리뷰:** 엘보우 그래프를 그리기 위한 for문 코드와, 최적의 k를 결정한 후 군집 결과를 시각화하는 과정을 단계별로 설명합니다.
3.  **'생각해 볼 문제' 토의:** 위 3가지 질문에 대한 팀의 답변을 발표합니다. 특히 PCA 데이터에 군집분석을 적용하는 이유(계산 효율성, 노이즈 감소, 시각화 용이성 등)를 명확히 설명하고, 이것이 분석의 질을 어떻게 높이는지 청중을 설득해 보세요.


## 문제 4 (난이도: 중): 고객 세그먼트 분석 및 전략 수립

> **🎯 목표:** 군집분석으로 얻은 그룹별 특성을 원본 데이터와 비교하여 분석하고, 각 그룹에 대한 페르소나를 정의하여 실제 비즈니스에 적용할 마케팅 아이디어를 도출합니다.


### 💡 핵심 개념:

분석의 최종 목표는 '인사이트 발견'과 '액션 제안'입니다. 군집분석으로 얻은 레이블(0, 1, 2...)은 그 자체로는 의미가 없습니다. 이 레이블을 원본 데이터프레임에 다시 붙인 후, 각 그룹이 원본 변수(소득, 소비점수 등)에서 어떤 평균적인 특징을 보이는지 분석해야 비로소 각 그룹의 정체를 파악하고, 'VIP 고객', '잠재 고객', '이탈 위험 고객' 등과 같은 의미 있는 이름을 붙여줄 수 있습니다.


### 📌 수행 과제:

1.  원본 `data` 데이터프레임에 군집 레이블(`cluster_labels`)을 새로운 컬럼으로 추가합니다.
2.  `groupby('cluster')`와 `.mean()`을 사용하여 각 군집별로 4개 원본 변수의 평균값을 계산합니다.
3.  계산된 평균값을 보고 각 군집의 특징을 해석하고, 각 군집에 어울리는 이름(페르소나)을 붙여봅니다. (예: '알뜰 쇼핑족', '쇼핑 중독 VIP' 등)
4.  각 고객 그룹의 특성에 맞는 마케팅 전략을 아이디어 차원에서 제안해봅니다.


In [ ]:
# 1. 원본 데이터에 군집 레이블 추가
# TODO-1: 원본 데이터프레임 'data'에 'cluster'라는 새로운 컬럼을 만들고, cluster_labels 값을 할당하세요.
data[___] = cluster_labels

# 2. 군집별 특성 분석
# TODO-2: 'cluster'로 그룹화(groupby)한 뒤, 평균(mean)을 계산하여 cluster_summary에 저장하세요.
cluster_summary = data.groupby(___).___()

print("군집별 평균 특성:")
print(cluster_summary)

# 3. 군집별 페르소나 정의 및 전략 수립 (아래 주석에 직접 작성해 보세요)
# Cluster 0: 어떤 특성을 가졌나요? (예: 소득은 높지만 소비는 낮은...)
#   - 페르소나:
#   - 마케팅 전략:

# Cluster 1: 어떤 특성을 가졌나요?
#   - 페르소나:
#   - 마케팅 전략:

# Cluster 2: 어떤 특성을 가졌나요?
#   - 페르소나:
#   - 마케팅 전략:

# (군집 개수에 따라 아래 내용을 채우세요)

### 🤔 생각해 볼 문제:

1.  이번 분석(PCA + K-Means)을 통해 얻은 가장 중요한 비즈니스 인사이트는 무엇이라고 생각하나요? 만약 이 분석을 하지 않았다면 어떤 점을 놓쳤을까요?
2.  K-평균 군집분석은 항상 원형의 군집을 잘 찾아내는 경향이 있습니다. 만약 데이터가 길쭉하거나 특이한 모양으로 분포해 있다면 K-평균이 적합하지 않을 수 있습니다. 이런 경우 어떤 다른 군집분석 방법을 고려해볼 수 있을까요? (힌트: `DBSCAN`, `Hierarchical Clustering`)
3.  이번 분석은 고객을 몇 개의 뚜렷한 그룹으로 나누었습니다. 하지만 실제 고객의 특성은 이렇게 칼로 자르듯 나뉘지 않을 수 있습니다. 이러한 '그룹화' 모델링의 장점과 현실 세계에 적용할 때의 잠재적 한계는 무엇일까요?

---

### 📢 **[발표 과제]**

문제 4를 맡은 팀은 **인사이트 도출 및 비즈니스 적용**의 전문가가 되어 다음 내용을 중심으로 발표를 준비하세요.

1.  **핵심 개념 설명:** 데이터 분석의 최종 목표가 '보고서 작성'이 아닌 '실질적 가치 창출'에 있음을 강조합니다. 군집 레이블을 다시 원본 데이터와 결합하여 해석하는 과정의 중요성을 설명하세요.
2.  **코드 리뷰 및 결과 해석:** `groupby().mean()` 코드의 의미를 설명하고, 출력된 요약 통계표를 어떻게 해석하여 각 군집의 특징을 잡아냈는지 상세히 보여줍니다. 각 군집에 붙인 창의적인 페르소나와 그 근거를 발표하세요.
3.  **종합 결론 및 토의:** 각 페르소나에 맞춰 제안한 마케팅 전략을 발표하며, 어떻게 데이터 분석 결과가 구체적인 비즈니스 액션으로 이어질 수 있는지 보여줍니다. '생각해 볼 문제' 토의를 통해, 이번 프로젝트 전체를 되돌아보고 데이터 기반 의사결정의 가치와 잠재적 한계에 대해 논의하며 전체 실습을 마무리하세요.
